In [1]:
import urllib.request
import folium
from pandas.io.json import json_normalize
import pandas as pd
from beakerx import *

In [2]:
url='https://raw.githubusercontent.com/annefou/jupyter_dashboards/gh-pages/data/Hardangervidda.geojson'
# Download the file from `url`, save it in a temporary directory and get the
# path to it (e.g. '/tmp/tmpb48zma.txt') in the `file_name` variable:
geojson_filename, headers = urllib.request.urlretrieve(url)

In [3]:
map = folium.Map(location=[60.5, 7.5], tiles='Stamen Terrain')

In [4]:
geojson = folium.GeoJson(
    geojson_filename,
    name='geojson'
).add_to(map)
features=geojson.data['features']

In [5]:
import json

def getStations(filename):
    data = json.load(open(filename))['features']
    stations={}
    for i in range(0,len(features)):   
        name = features[i]['properties']['description']
        opr = int(features[i]['properties']['waspmote_id'],16)
        stations[name] = opr
    return (stations)

In [6]:
stations=getStations(geojson_filename)
print(stations)

{'Appelsinhytta': 161398434909148276, 'Hills': 7383483387351159950, 'Middaselvi discharge': 4574644594755523609, 'Finselvi discharge': 4634607560887915575, 'Drift lower lidar': 7873531321804412124}


In [7]:
import requests
URL = 'http://hycamp.org/wsn/api/query/'
token = 'dcff0c629050b5492362ec28173fa3e051648cb1'
headers = {'Authorization': 'Token %s' % token}

In [8]:
def getFinseStation(station, sensor):
        params  = {
            'limit': 10000,
            'offset': 0,
            'mote': station,
            'xbee': None,
            'sensor': sensor,
            'tst__gte': None,
            'tst__lte': None,
            }
        r = requests.get(URL, headers=headers, params = params)
        data = r.json()['results']
        if r.status_code == 200:
            FinseStations = json_normalize(data)       
            FinseStations['timestamp'] = pd.to_datetime(FinseStations.epoch, unit='s')
            return FinseStations
        else:
            print("Data not available")
            return None

In [9]:
def getSensors(mote):
    params  = {
            'limit': 10000,
            'offset': 0,
            'mote': mote,
            'xbee': None,
            'sensor': None,
            'tst__gte': None,
            'tst__lte': None,
            }
    r = requests.get(URL, headers=headers, params = params)
    data = r.json()['results']
    if r.status_code == 200:
        FinseStations = json_normalize(data)
        listSensors=FinseStations['sensor'].unique()
        return listSensors
    else:
        print("Data not available")
        return ([],[])
    

In [10]:
sensors={}
for station in stations:
    print(station)
    sensors[station]=getSensors(stations[station])
print(sensors)

Appelsinhytta
Hills
Middaselvi discharge
Finselvi discharge
Drift lower lidar
{'Appelsinhytta': array(['ds2_meridional', 'ds2_dir', 'ds2_zonal', 'ds2_speed', 'ds2_temp',
       'bat', 'ds2_gust'], dtype=object), 'Hills': array(['rssi', 'ds2_speed', 'bat', 'ds2_gust', 'ds2_temp', 'ds2_zonal',
       'ds2_meridional', 'ds2_dir'], dtype=object), 'Middaselvi discharge': array(['bat'], dtype=object), 'Finselvi discharge': array(['ctd_temp', 'bat', 'ctd_depth', 'ctd_cond'], dtype=object), 'Drift lower lidar': array(['rssi', 'ds2_zonal', 'ds2_temp', 'ds2_speed', 'bat', 'ds2_dir',
       'ds2_gust', 'ds2_meridional'], dtype=object)}


In [11]:
import ipywidgets as widgets
from IPython.display import display, HTML


def plot_sensor(station,sensor):
    station_id=stations[station]
    FinseStation = getFinseStation(station_id, sensor)
    if sensor == 'bat':
        print("BAT histogram")
    else:
    # simple timeserie plot
        # Select a sensor: for instance ds2_temp
        ds2_temp = FinseStation.loc[FinseStation['sensor'] == sensor]
        # Use BeakerX to plot:
        tsplot = TimePlot(title="Title to adapt",
         xLabel="Date",
         yLabel="Whatever...")
        tsplot.add(Line(x=ds2_temp['timestamp'], y=ds2_temp['value'], color=Color.DARK_GRAY))
        display(tsplot)
        # Select a sensor: for instance ds2_temp

def select_sensor(change):
    print(change)
    sensorW.options = sensors[change['new']]

def print_values(val):
    print(val)


stationW = widgets.Select(options=sensors.keys(), description='Stations')
init = stationW.value
sensorW = widgets.RadioButtons(options=sensors[init], description='Sensors')
stationW.observe(select_sensor, names="value")
interact_plot = widgets.interact_manual(plot_sensor, 
                    station = stationW,
                    sensor= sensorW );
interact_plot.widget.children[-1].layout.height = '600px'   #set the height of the interactive plot so it won't change size
interact_plot.widget.children[2].description="Click to plot"
display(interact_plot)


interactive(children=(Select(description='Stations', options=('Appelsinhytta', 'Hills', 'Middaselvi discharge', 'Finselvi discharge', 'Drift lower lidar'), value='Appelsinhytta'), RadioButtons(description='Sensors', options=('ds2_meridional', 'ds2_dir', 'ds2_zonal', 'ds2_speed', 'ds2_temp', 'bat', 'ds2_gust'), value='ds2_meridional'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.plot_sensor>